### Lecture 5: Gradient Descent,  Basic NN Architecture, Activation Functions

#### Hinge Loss Review

- Binary setting: $h(y^i, x^i) = max(0, 1 - y^iw^Tx^i)$. 
- This means that if $sign(y^i) = sign(x^i)$ then if $\mid{} w^Tx^i \mid{} > 1$ then the loss is 0. Otherwise the loss is in between 0 and 1. 
- On the other hand, if the signs differ, then the loss is greater than 1 (greater penelization for an incorrect prediction, less penelization for correct but not confident predictions). 
- Multi-class hinge loss: $h(y^i, x^i) = \sum_{j != y^i}^{C} \max(0, 1 + w_j^Tx^i - w_{y_i}^Tx^i)$. 
- Interpretation: if the correct class score is greater than the incorrect class score by some margin $\Delta$, here set to $1$, then the loss is 0. If the correct class score is greater by some $\Delta < 1$ then the loss is in between $0$ and $1$. 

#### Gradient Descent
- Main idea: We can't generally find closed-form optima, so iteratively approach the optimum via gradient descent instead. 
- For $x \in R^{n}$ the gradient is a vector whose $i$th element is the partial derivative: $$\nabla_x f(x) = \begin{bmatrix} \frac{df(x)}{dx_1} \\ \frac{df(x)}{dx_2} \\ ... \end{bmatrix}$$
- Gradient tells us how a small $\delta x$ affects $f(x)$: $f(x + \delta x) = f(x) + \nabla_x f(x)$. 

#### Derivation of "Steepest Descent"
- $u^T \nabla_xf(x)$ gives the derivative of $f(x)$ in the direction given by the unit vector $u$. 
- We have to find the direction in which $f(x)$ decreases fastest. This is $$\min_u u^T\nabla_x f(x) = \min_u \nabla_x f(x) \cos(\theta)$$
- We set $\cos \theta = -1$, meaning that $u$ is pointing in the opposite direction of the gradient.
- This gives us our gradient descent, or "steepest descent" update rule: $x:= -\epsilon \nabla_x f(x)$. 

#### Why not use a numerical gradient? 
- The numerical gradient would be obtained by doing something like $$\frac{df(W)}{w_i} = \lim_{h \rightarrow{} 0} \frac{f(W - \Delta w_i) - f(W + \Delta w_i)}{2h}$$
- Generally models have a lot of parameters, and neural networks can have millions of parameters, so doing this many forward propagations (basically $O(n)$ in the number of parameters) to get all of the updates will take forever
- Also it's an approximation, and an incorrect gradient can totally mess up the learning process
- Much better to do backpropagation, which is more expensive than a single forward pass, but we get all of the gradients with backprop.

#### Hinge Loss Subgradient Example
- Given the hinge loss $h(y^i, x^i) = \sum_{j != y^i}^{C} \max(0, 1 + w_j^Tx^i - w_{y_i}^Tx^i)$, we can write down the (sub) gradients with respect to the weighst and biases. For $w_j, j \neq y_i$, we have $\nabla_{w_j} L_i = \mathbb{1}(1 + w_j^Tx_i - w_{y_i}^Tx_i \geq 0)x_i $, and for $w_{y_i}$ we have, since there is a $w_{y_i}$ in each term of the inner sum, $\nabla_{w_{y_i}}L_i = \sum_{j != y_i} \mathbb{1}( + w_jx^T_i - w_{y_i}^Tx^i  > 0)(-x^i)$


#### Neural Network Architecture
![nn architecture](nnarchitecture.png)
- "Fully connected" architecture -> every neuron is connected to all of the neurons in the next layer by some weights
- A neuron computes $f(\sum_i w_ix_i + b)$ where $f$ is some nonlinearity

#### Activation Functions


- Sigmoidal activation: $\sigma(x) = \frac{1}{1 + \exp(-x)}$.
- Derivative is $\sigma(x)(1 - \sigma(x))$. 
- It behaves linearly around $x = 0$, and is everywhere differentiable. Also has the interpretation of a probability since the outputs are between $0$ and $1$, and also gives the "firing rate" of the neuron.
- Cons: Saturating gradient at extremes -> gradient is practically 0.
- max value of the gradient is $0.25$, so it also always scales down gradients as they are backpropagated through, which also can lead to the vanishing gradient problem. 
- Another con is that it is centered around $0.5$, not $0$. This means that all of the outputs from the sigmoid neuron will be positive, and centered around $0.5$. This could result in "zig-zagging" during gradient descent. For concreteness, suppose that $f = \sum_i w_ix_i$ and $\frac{df}{dw_i} = x_i$. Then if we have a loss $L = \sum_i y^i - f(x^i)$, we have $\frac{dL}{dw_i} = \frac{dL}{df}\frac{df}{dw_i} = \frac{dL}{df}x_i$ This tells us that the gradient of the loss with respect to the weights will always have the same sign, since $x_i$ is positive, the gradients will either be all positive or all negative, resulting in "zig zagging" during gradient descent.  
- Hyperbolic Tanh: $2\sigma(x) - 1$, $\frac{d}{dx}tanh(x) = 1 - tanh^2(x)$> 
- This is still saturating but less so than the sigmoid, and behaves as a nearly linear unit for more of it's input range. Also has the property of beign zero-centerred, which means that there will not be much zig-zagging of the weight updates during gradient descent (see above for a more detailed description of this). 

#### The Humble ReLU, and it's several variations
- ReLU unit is given by $ReLU(x) = max(0, x)$. The local gradient is $1$ if $x > 0$ else $0$. The unit is not differentiable at $0$, but we generally assign a value to the derivative at $0$ between $0, 1$. 
- The pros of the relu unit include faster convergence, behaves linearly when it's active, and does not scale gradients down during backpropagation, it rather just "passes them through" the unit. 
- However, learning does not happen for examples with $0$ activation, which can lead to the "dying ReLU" problem which is when a ReLU neuron never activates, such as if its weights are not initialized properly or if a gradient update knocks them in a certain direction (literally what [Karpathy's post](https://medium.com/@karpathy/yes-you-should-understand-backprop-e2f06eab496b) says). 
- The softplus unit $\log(1 + \exp(x))$ is a soft approximation of the ReLU who's derivative is the sigmoid function. 
- The ** leaky relu** seeks to solve teh dying ReLU problem by passing a smaller gradient when $x < 0$ and the unit is not active in teh forward pass. It is given by $\max(\alpha x, x)$ where $\alpha$ is a hyperparameter generally set to some small value like $0.01$. In the similar PReLU, this $\alpha$ is actually a parameter that can be optimized during learning.
- The exponential linear unit, or ELU, is another activation function that's designed to approximate the ReLU but also avoid teh stopping of learning when the unit is not active. It is given by $f(x) = \max(\alpha(\exp(x) - 1), x)$. 

#### Output Activations
- ONce we have the output socres form our network, we may want to run them through an activation function before calculating our loss or for interpretability purposes. 
- A popular output activation is the sigmoid or more general softmax, which allows our scores to be interpreted as probabilities. 
- Based on this, which loss function may be better to use? The **MSE** given by $L = \frac{1}{2}\sum_i (y^i - \sigma(z^i))$ or the cross entropy given by $l = -\sum_i y_i \log \sigma(z^i) + (1 - y_i)\log(1 - \sigma(z^i))$.
- If we consider the backwards pass, the derivatives are given by $-2(y_i - \sigma(z_i))\sigma'(z_i)$, so if $z$ is very large and negative or very large and positive, this gradient will be approximately zero. This isn't good because it implies that we have a lages loss and an inaccurate prediciton, but essentially zero gadient will be passed down to our weights, leading to extremely slow or nearly no learning (essentially the vanishing gradient problem again). 
- On the other hand, the derivative for the cross entropy cost can be given by $\frac{dCE}{dz_i} = \sigma(z_i) - 1$, which is not dependent on the sigmoid derivative. In this case when $z$ is negative and large, then the gradient will have the highest magnitude, and it will only go towards zero when $z$ is positive and large, implying that we have a low loss, which makes sense. 


